In [1]:
from alexnet.helper import *
from alexnet.alexnet import *
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
from tqdm import tnrange,tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

In [2]:
epochs = 2
batch = 16 # batch of Images
batch_size = 16

logs_path = "tensorflow_logs/alexfish"
model_path  = "models/alex"

In [3]:
img_list = img_location_reader("data")
train,test = train_test_split(img_list,test_size=0.3,random_state=0)
train,valid = train_test_split(train,test_size=0.1,random_state=0)

In [4]:
x_dev = img_location_list(train,over_sample=True)
print (len(x_dev))
x_val = img_location_list(valid,over_sample=False)
print (len(x_val))

9224
274


In [5]:
img_dummy_label = pd.get_dummies(list(image_label(x_dev).keys()))

In [6]:
X = tf.placeholder(tf.float32,[None,224,224,3],name = "Input_data")
Y = tf.placeholder(tf.float32,[None,8], name = "InputLabels")



weights ={
"w1":init_weights([11,11,3,96]),
"w2":init_weights([5,5,96,256]),
"w3":init_weights([3,3,256,384]),
"w4":init_weights([3,3,384,384]),
"w5":init_weights([3,3,384,256]),
"w6":init_weights([4096,4096]),
"w7":init_weights([4096,4096]),
"w8":init_weights([4096,8])
}

bias ={
"b1":init_bias_zero([96]),
"b2":init_bias_ones([256]),
"b3":init_bias_zero([384]),
"b4":init_bias_ones([384]),
"b5":init_bias_ones([256]),
"b6":init_bias_ones([4096]),
"b7":init_bias_ones([4096])
}

In [7]:
pred = alexnet(X,weights,bias)

In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,Y))
#optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(cost)
optimizer = tf.train.RMSPropOptimizer(0.001,decay=0.9,momentum=0.9,epsilon=1e-10).minimize(cost)

In [9]:
#Evaluate the model
correct_pred= tf.equal(tf.argmax(pred,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

#Initialize all the variables
init = tf.initialize_all_variables()

saver = tf.train.Saver()

In [10]:
# Create summary to monitor cost tensor , accuracy tensor
tf.scalar_summary("loss",cost)
tf.scalar_summary("accuracy",accuracy)

merged_summary_op = tf.merge_all_summaries()

sudo code 

- initalize all the paramaters 
- for each epoch 
      - x_dev contains filelocations , shuffle them 
      - take a batch of 16 images 
      - read the images (alex_dev_image_reader aguments the data and makes them 160 images), these are also shuffled
          - now feed 16(batch_size) images at once to the network
      - now for every 100 iterations , calculate validation accuracy and append it to val_acc
      - save the model 
      - continue till all the epochs are finished 

In [11]:
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.train.SummaryWriter(logs_path,graph=tf.get_default_graph())
    val_acc = []
    for ep in tqdm_notebook(range(epochs),desc="1st loop"):
        print ("epoch: ", ep)
        img_list = shuffle(x_dev,random_state=0)
        for i in tqdm_notebook(range(int(len(img_list)/batch)),desc="2nd loop",leave=False):
            img_loc = img_list[i*batch:(i+1)*batch]
            X_image, Y_image =alex_dev_image_reader(img_loc,img_dummy_label,size=(256,256),normalize = True)
            for k in range(int(len(X_image)/batch_size)):
                x_image,y_image = X_image[k*batch_size:(k+1)*batch_size],Y_image[k*batch_size:(k+1)*batch_size]
                _,c,summary = sess.run([optimizer,cost,merged_summary_op],feed_dict={X : np.float32(x_image), Y : np.float32(y_image)})
            train_loss, train_acc = sess.run([cost,accuracy], feed_dict={X : np.float32(X_image), Y : np.float32(Y_image)})
            if i % 100 == 0:
                x,y = [],[]
                for j in tqdm_notebook(range(len(x_val)),desc="3rd loop",leave=False):
                    xx_val,y_val =alex_val_image_reader(x_val[j],img_dummy_label,size=(256,256),normalize = True)
                    value = sess.run([tf.nn.softmax(pred)], feed_dict={X : np.float32(xx_val)})
                    x.append(np.average(value[0], axis=0))
                    y.append(np.average(y_val, axis=0))
                x,y = np.concatenate(x).reshape(-1,8),np.concatenate(y).reshape(-1,8)
                valid_acc = sess.run([tf.reduce_mean(tf.cast(tf.equal(tf.argmax(x,1),tf.argmax(Y,1)),tf.float32))],\
                        feed_dict={Y : np.float32(y)})
                print (str(ep),str(i),"Train loss ="+"{:.6f}".format(train_loss),\
                       "Train acc ="+"{:.6f}".format(train_acc),\
                      "valid acc ="+"{:.6f}".format(valid_acc[0]))
                val_acc.append(valid_acc[0])
                saver.save(sess,os.getcwd()+model_path+"_"+str(ep)+"_"+str(i))
    print("Optimization Finished!")


epoch:  0
0 0 Train loss =2.249001 Train acc =0.150000 valid acc =0.193431
0 100 Train loss =12.092441 Train acc =0.156250 valid acc =0.343066
0 200 Train loss =12.366575 Train acc =0.156250 valid acc =0.105839
0 300 Train loss =26.927835 Train acc =0.181250 valid acc =0.051095
0 400 Train loss =29.662460 Train acc =0.306250 valid acc =0.021898
0 500 Train loss =18.879187 Train acc =0.168750 valid acc =0.193431
epoch:  1
1 0 Train loss =26.906130 Train acc =0.131250 valid acc =0.135036
1 100 Train loss =19.073000 Train acc =0.206250 valid acc =0.040146
1 200 Train loss =20.716362 Train acc =0.125000 valid acc =0.120438
1 300 Train loss =24.445259 Train acc =0.200000 valid acc =0.072993
1 400 Train loss =16.982250 Train acc =0.268750 valid acc =0.029197
1 500 Train loss =16.125187 Train acc =0.175000 valid acc =0.032847

Optimization Finished!


In [ ]:

# with tf.Session() as sess:
#     sess.run(init)
#     x, y = [],[]
#     for i in tqdm(range(len(x_val))):
#         xx_val ,y_val = alex_val_image_reader(x_val[i],img_dummy_label,size=(256,256),normalize = True)
#         value = sess.run([tf.nn.softmax(pred)], feed_dict={X : np.float32(xx_val)})
#         x.append(np.average(value[0], axis=0))
#         y.append(np.average(y_val, axis=0))
#     x,y = np.concatenate(x).reshape((-1,8)),np.concatenate(y).reshape((-1,8))
#     acc = sess.run([tf.reduce_mean(tf.cast(tf.equal(tf.argmax(x,1),tf.argmax(Y,1)),tf.float32))],\
#                         feed_dict={Y : np.float32(y)})
#     print (acc)

In [15]:
epochs = 10
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess,os.getcwd()+model_path+"_"+str(1)+"_"+str(400))
    summary_writer = tf.train.SummaryWriter(logs_path,graph=tf.get_default_graph())
    val_acc = []
    for ep in tqdm_notebook(range(epochs),desc="1st loop"):
        print ("epoch: ", ep)
        img_list = shuffle(x_dev,random_state=0)
        for i in tqdm_notebook(range(int(len(img_list)/batch)),desc="2nd loop",leave=False):
            img_loc = img_list[i*batch:(i+1)*batch]
            X_image, Y_image =alex_dev_image_reader(img_loc,img_dummy_label,size=(256,256),normalize = True)
            for k in range(int(len(X_image)/batch_size)):
                x_image,y_image = X_image[k*batch_size:(k+1)*batch_size],Y_image[k*batch_size:(k+1)*batch_size]
                _,c,summary = sess.run([optimizer,cost,merged_summary_op],feed_dict={X : np.float32(x_image), Y : np.float32(y_image)})
            train_loss, train_acc = sess.run([cost,accuracy], feed_dict={X : np.float32(X_image), Y : np.float32(Y_image)})
            if i % 100 == 0:
                x,y = [],[]
                for j in tqdm_notebook(range(len(x_val)),desc="3rd loop",leave=False):
                    xx_val,y_val =alex_val_image_reader(x_val[j],img_dummy_label,size=(256,256),normalize = True)
                    value = sess.run([tf.nn.softmax(pred)], feed_dict={X : np.float32(xx_val)})
                    x.append(np.average(value[0], axis=0))
                    y.append(np.average(y_val, axis=0))
                x,y = np.concatenate(x).reshape(-1,8),np.concatenate(y).reshape(-1,8)
                valid_acc = sess.run([tf.reduce_mean(tf.cast(tf.equal(tf.argmax(x,1),tf.argmax(Y,1)),tf.float32))],\
                        feed_dict={Y : np.float32(y)})
                print (str(ep),str(i),"Train loss ="+"{:.6f}".format(train_loss),\
                       "Train acc ="+"{:.6f}".format(train_acc),\
                      "valid acc ="+"{:.6f}".format(valid_acc[0]))
                val_acc.append(valid_acc[0])
                saver.save(sess,os.getcwd()+model_path+"_"+str(ep)+"_"+str(i))
    print("Optimization Finished!")

epoch:  0
0 0 Train loss =20.347353 Train acc =0.200000 valid acc =0.463504
0 100 Train loss =17.304625 Train acc =0.137500 valid acc =0.299270
0 200 Train loss =26.554459 Train acc =0.131250 valid acc =0.419708
0 300 Train loss =19.559610 Train acc =0.206250 valid acc =0.032847
0 400 Train loss =18.601959 Train acc =0.218750 valid acc =0.098540
0 500 Train loss =21.811127 Train acc =0.168750 valid acc =0.091241
epoch:  1
1 0 Train loss =28.732042 Train acc =0.162500 valid acc =0.113139
1 100 Train loss =16.210882 Train acc =0.162500 valid acc =0.182482
1 200 Train loss =17.446327 Train acc =0.275000 valid acc =0.025547
1 300 Train loss =19.469185 Train acc =0.175000 valid acc =0.098540
1 400 Train loss =21.272594 Train acc =0.343750 valid acc =0.014599
1 500 Train loss =23.805321 Train acc =0.237500 valid acc =0.043796
epoch:  2
2 0 Train loss =24.381409 Train acc =0.125000 valid acc =0.054745
2 100 Train loss =22.508644 Train acc =0.175000 valid acc =0.149635
2 200 Train loss =15.078

In [ ]:
# Try increasing the validation set - 0.1 to 0.3 
# decrease the learning rate
# use L2 regularization